In [50]:
import folium
import requests

import os
import json

from selenium import webdriver
from xvfbwrapper import Xvfb

import time 

# 
SCRAPING_INTERVAL = 60 * 1   # seconds
COUNTS = 100

In [53]:

def create_png(arr, tid, display=None):

    if display:
        print('in virtual display')
        display.start()

    # Ekat
    LAT = 56.8518982
    LON = 60.6122017
    col_tram = '#d2444c'  # LAYER 1
    col_trol = '#3186cc'  # LAYER 2

    m = folium.Map(location=[LAT, LON], zoom_start=12)
    temp = arr[0].get('ATIME')
    print('TEMP ', temp)
    folium.Marker(location=[LAT+0.08, LON-0.03],
                  # Set icon to DivIcon to with conditional style formatting to reference the random temp value.
                  icon=folium.DivIcon(
                      html=f"""<div style="font-family: courier new; color: 'lightblue';background: #ccc;font-weight: 700; display: inline-block;
white-space: nowrap;">{"{}".format(temp)}</div>""")
                 ).add_to(m)
    for t in arr:
        folium.Circle(
            radius=5,
            location=[t.get('LAT'), t.get('LON')],
            color=col_tram if t.get('LAYER') == '1' else col_trol,
            fill=True,
            fill_color=col_tram if t.get('LAYER') == '1' else col_trol,
        ).add_to(m)
    fn='testmap.html'
    m.save(fn)
    print(f'saved {fn}')
    delay=5
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

    browser = webdriver.Chrome()
    browser.get(tmpurl)
    #Give the map tiles some time to load
    time.sleep(delay)
    browser.save_screenshot(f'map_{tid}.png')
    print(f'save_screenshot map_{tid}.png')

    browser.quit()
    
    if display:
        display.stop()
    m

In [54]:
import sched, time, datetime 
from IPython.lib import backgroundjobs as bg
jobs = bg.BackgroundJobManager()

display = Xvfb(width=1920, height=1080)

def get_data(n):
    # 'http://map.ettu.ru/api/v2/troll/boards/?apiKey=111&order=1'
    sres = []
    for type_tran in ['troll', 'tram']:
        res = requests.get(
                f'http://map.ettu.ru/api/v2/{type_tran}/boards/?apiKey=111&order=1'
            )
        sres = sres + res.json().get('vehicles')
    create_png(sres, n, display)
            
def printfunc(interval=1, reps=5):
    for n in range(reps):
        num = n + 1
        print(f'start {datetime.datetime.now()}... {num}')
        get_data(num)
        time.sleep(interval)

    print('All done!')

    
print(f'first run at({datetime.datetime.now()})   and run jobs with interval {fint} sek and repeats {frep}')
printfunc(SCRAPING_INTERVAL, COUNTS)
    
# jobs.new(printfunc, fint, frep)
# jobs.status()




first run at(2020-01-21 15:52:34.147621)   and run jobs with interval 60 sek and repeats 100
start 2020-01-21 15:52:34.147729... 1
in virtual display
TEMP  2020-01-21 15:52:09
saved testmap.html
save_screenshot map_1.png
start 2020-01-21 15:53:51.382167... 2
in virtual display
TEMP  2020-01-21 15:53:24
saved testmap.html
save_screenshot map_2.png
start 2020-01-21 15:55:01.861863... 3
in virtual display
TEMP  2020-01-21 15:54:09
saved testmap.html
save_screenshot map_3.png
start 2020-01-21 15:56:12.909447... 4
in virtual display
TEMP  2020-01-21 15:55:54
saved testmap.html
save_screenshot map_4.png
start 2020-01-21 15:57:24.112859... 5
in virtual display
TEMP  2020-01-21 15:57:09
saved testmap.html
save_screenshot map_5.png
start 2020-01-21 15:58:34.694887... 6
in virtual display
TEMP  2020-01-21 15:58:09
saved testmap.html
save_screenshot map_6.png
start 2020-01-21 15:59:45.413502... 7
in virtual display
TEMP  2020-01-21 15:59:09
saved testmap.html
save_screenshot map_7.png
start 2020-

saved testmap.html
save_screenshot map_63.png
start 2020-01-21 17:07:06.735111... 64
in virtual display
TEMP  2020-01-21 17:05:31
saved testmap.html
save_screenshot map_64.png
start 2020-01-21 17:08:18.271303... 65
in virtual display
TEMP  2020-01-21 17:07:31
saved testmap.html
save_screenshot map_65.png
start 2020-01-21 17:09:28.486346... 66
in virtual display
TEMP  2020-01-21 17:08:46
saved testmap.html
save_screenshot map_66.png
start 2020-01-21 17:10:39.146885... 67
in virtual display
TEMP  2020-01-21 17:10:16
saved testmap.html
save_screenshot map_67.png
start 2020-01-21 17:11:49.426261... 68
in virtual display
TEMP  2020-01-21 17:11:31
saved testmap.html
save_screenshot map_68.png
start 2020-01-21 17:12:59.718266... 69
in virtual display
TEMP  2020-01-21 17:12:31
saved testmap.html
save_screenshot map_69.png
start 2020-01-21 17:14:10.233921... 70
in virtual display
TEMP  2020-01-21 17:13:31
saved testmap.html
save_screenshot map_70.png
start 2020-01-21 17:15:21.215665... 71
in vi

In [ ]:
# bash 
# standart create gif
# convert -delay 10 -loop 0 *.png anim.gif
# Some compress
!mkdir cropped
!for file in *.png; do convert $file -resize 840x680 -quality 95 cropped/$file; done 

# Create gif with ffmpeg
# create pallete 
!ffmpeg -i ./cropped/map_%d.png -vf palettegen ./cropped/palette.png

# create gif 
!ffmpeg -i ./cropped/map_%d.png -i ./cropped/palette.png -lavfi "paletteuse,setpts=6*PTS" -y out.gif


In [56]:
from IPython.display import HTML
HTML('<img src="out.gif">')
